In [71]:
import os
import pandas as pd
import spacy

In [72]:
workdir = 'drive/MyDrive/Deep_Learning/NLP/hero_vired_assignment'


In [73]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [74]:
!pip install redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
import random
import numpy as np
import pandas as pd
import time
from redis import Redis
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
from redis.commands.search.result import Result

In [76]:
data = pd.read_csv(workdir+'/data/database.csv')

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  10001 non-null  object
 1   text      10001 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [78]:
data.duplicated().value_counts()

False    10001
dtype: int64

In [79]:
# host = 'vecsim'
# port = 6379
# redis_conn = Redis(host = host, port = port)
# print ('Connected to redis')

In [80]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
# Extract the questions from the dataset
questions = data['question'].tolist()

In [82]:
# Step 2: Vectorization
from sentence_transformers import SentenceTransformer

# Load the pre-trained embedding model
# model = SentenceTransformer('bert-base-nli-mean-tokens')


In [83]:
# Vectorize the dataset questions
dataset_question_vectors = model.encode(questions)


In [84]:
dataset_question_vectors

array([[-0.00694904,  0.03551034, -0.00165744, ..., -0.00651182,
         0.02655866, -0.01957088],
       [-0.04021484,  0.03987261,  0.02234971, ...,  0.02597052,
        -0.01058564,  0.01428081],
       [-0.00830854,  0.02165501,  0.01550877, ..., -0.0230731 ,
        -0.00137292,  0.00796463],
       ...,
       [-0.0383231 ,  0.02456209,  0.02822157, ...,  0.022478  ,
         0.00320681, -0.03160231],
       [ 0.00608045,  0.00958769, -0.01674117, ...,  0.03812591,
        -0.02920343,  0.06175386],
       [-0.00781723, -0.00033367,  0.02213507, ...,  0.03068331,
        -0.07054033, -0.00951812]], dtype=float32)

In [85]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
# Step 3: Vector Database
from annoy import AnnoyIndex

In [87]:
# Create an Annoy index for vector storage
vector_dimension = len(dataset_question_vectors[0])
annoy_index = AnnoyIndex(vector_dimension, 'angular')

In [88]:
# Add the dataset question vectors to the Annoy index
for i, vector in enumerate(dataset_question_vectors):
    annoy_index.add_item(i, vector)

In [89]:
# Build the Annoy index
annoy_index.build(10)

True

In [102]:
# Step 4: Query Processing
user_question = "car engine optimal temperature"

# Vectorize the user's question
user_question_vector = model.encode([user_question])[0]

In [103]:
# Search the user's question against the dataset questions in the Annoy index
# step 5:
similar_question_indices = annoy_index.get_nns_by_vector(
    user_question_vector, 5, search_k=-1, include_distances=False
)

In [104]:
# Step 6: Retrieve Top 5 Similar Questions
similar_questions = [questions[idx] for idx in similar_question_indices]

In [105]:
# Step 7: Display the Results
print("Top 5 Similar Questions:")
for i, question in enumerate(similar_questions, 1):
    print(f"{i}. {question}")

Top 5 Similar Questions:
1. Why is it recommended we warmup our vehicles on colder days?
2. Why do (a lot of) cars get optimal fuel efficiency at 55-60 mph/88-95 kph??
3. New irons which have no variable temperature setting
4. Why is 350 degrees farenheit the most common temperature for most recipes?
5. Why do engines/motors sometimes have a hard time starting up in colder weather?
